In [1]:
import glob
from key import *
from langchain.docstore.document import Document
from loader import Loader

In [2]:
loader = Loader(PATH)
documents = loader.load()

In [11]:
text = open('data/Clean/index.md', "r").readlines()

In [12]:
text = "".join(text)

In [13]:
text

'# Home\n\n## Introduction\n\nAOS ("Autonomous Operating System") is an open source event-loop based framework for\nrealtime robotic systems. It is meant to serve the same role as\n[ROS](https://www.ros.org/) in a robotic system (including the fact that, like\nROS, it is not actually an Operating System). However, the design priorities of\nAOS are somewhat different from those of both ROS and ROS2. See the [Design of\nAOS](design) for more detail.\n\nBecause AOS was originally developed under the auspices of a [High School\nRobotics Team](https://frc971.org/), it is still hosted on the corresponding\nservers, and that the code we refer to as "AOS" is present in the\n[aos](https://github.com/frc971/971-Robot-Code/tree/master/aos) folder.\n\n## Contributing\n\n[971-Robot-Code](https://github.com/frc971/971-Robot-Code)\nis the canonical GitHub mirror of the\nAOS code. Issues or feature requests can be reported using the GitHub issues page\n(this is subject to change); however, contributio

In [16]:
doc =  Document(page_content=text,         metadata = {
            "type" : "markdown",
            "path" : 'data/Clean/index.md'
        })

In [17]:
doc

Document(page_content='# Home\n\n## Introduction\n\nAOS ("Autonomous Operating System") is an open source event-loop based framework for\nrealtime robotic systems. It is meant to serve the same role as\n[ROS](https://www.ros.org/) in a robotic system (including the fact that, like\nROS, it is not actually an Operating System). However, the design priorities of\nAOS are somewhat different from those of both ROS and ROS2. See the [Design of\nAOS](design) for more detail.\n\nBecause AOS was originally developed under the auspices of a [High School\nRobotics Team](https://frc971.org/), it is still hosted on the corresponding\nservers, and that the code we refer to as "AOS" is present in the\n[aos](https://github.com/frc971/971-Robot-Code/tree/master/aos) folder.\n\n## Contributing\n\n[971-Robot-Code](https://github.com/frc971/971-Robot-Code)\nis the canonical GitHub mirror of the\nAOS code. Issues or feature requests can be reported using the GitHub issues page\n(this is subject to change)

In [4]:
my_list = ["Hello", " ", "world", "!"]
result_string = "".join(my_list)
print(result_string)

Hello world!


In [5]:
glob.glob('data/**/*.*', recursive=True)

['data/Clean/download-code-to-the-robot.md',
 'data/Clean/reference.md',
 'data/Clean/README2.md',
 'data/Clean/README3.md',
 'data/Clean/setup-ssh-vscode.md',
 'data/Clean/frc971.conf',
 'data/Clean/allwpilib_ni-libraries_tarball.sh',
 'data/Clean/README.md',
 'data/Clean/mkdocs_bin.py',
 'data/Clean/getting_started.md',
 'data/Clean/make_bazel_package.sh',
 'data/Clean/index.md',
 'data/Clean/submitting-code-for-a-review.md',
 'data/Clean/setup-pycharm-on-build-server.md',
 'data/Clean/allwpilib_subtree.sh',
 'data/manuals/2022FRCGameManual.pdf',
 'data/manuals/2023FRCGameManual.pdf',
 'data/manuals/2019FRCGameSeasonManual.pdf',
 'data/File Dump Dec 4/frc-volunteer-roles.pdf',
 'data/File Dump Dec 4/1796.pdf',
 'data/File Dump Dec 4/3654 (1).pdf',
 'data/File Dump Dec 4/1741.pdf',
 'data/File Dump Dec 4/772.pdf',
 'data/File Dump Dec 4/emergency-response-template.pdf',
 'data/File Dump Dec 4/4598.pdf',
 'data/File Dump Dec 4/4201.pdf',
 'data/File Dump Dec 4/5985 (1).pdf',
 'data/Fil